## Get site image

In [3]:
import requests
import json
import re
import csv

In [20]:
def get_osm_id(address):
    print(address)
    base_url = "https://nominatim.openstreetmap.org/search"
    params = {
        'q': address,
        'format': 'json',
        'limit': 1
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data:
            center_coordinate = float(data[0].get("lon")), float(data[0].get("lat"))
            osm_id = data[0]['osm_id']
            return osm_id, center_coordinate
        else:
            print("No results found for the address.")
            return None
    else:
        print("Error:", response.status_code)
        return None

def get_coordinates_of_way(osm_id):
    overpass_url = "http://overpass-api.de/api/interpreter"
    way_query = f"""
    [out:json];
    way({osm_id});
    out geom;
    """

    try:
        response = requests.get(overpass_url, params={'data': way_query})
        response.raise_for_status()
        way_data = response.json()
        if not way_data['elements']:
            print(f"Way {osm_id} not found!")
            return []

        coords = way_data['elements'][0]['geometry']
        return [(point['lat'], point['lon']) for point in coords]

    except requests.RequestException as e:
        print(f"Error fetching data for way {osm_id} from Overpass API: {e}")
        return []


def coordinates_to_geojson(coordinates, osm_type='way'):
    if not coordinates or coordinates[0] != coordinates[-1]:
        print("The provided coordinates do not form a closed loop. Cannot create a Polygon.")
        return None

    # Flip the coordinates from (lat, lon) to (lon, lat)
    flipped_coordinates = [[lon, lat] for lat, lon in coordinates]
    
    return flipped_coordinates

def main(address):
    osm_id, center_coordinates = get_osm_id(address)
    if osm_id is None:
        return
    location = get_coordinates_of_way(osm_id)
    geojson = coordinates_to_geojson(location)
    return osm_id, center_coordinates, location, geojson

address = 'simei green'
_, center_coord, _, geojson = main(address)
print(center_coord)
print(geojson)

geojson_feature = {
    "type": "Feature",
    "properties": {},
    "geometry": {
        "type": "Polygon",
        "coordinates": [geojson]
    }
}
geojson_str = json.dumps(geojson_feature)
encoded_geojson = requests.utils.quote(geojson_str)


# Replace the placeholder values with your actual data
access_token = 'sk.eyJ1IjoieWVuY2hpbmdwIiwiYSI6ImNsb3FrZ2U1czBqZjkycWx3cXVlbDRzOXkifQ.hTjiIfuNE7I-KhYqJfQkFw'  # Replace with your Mapbox access token
lon, lat = center_coord  # Replace with the center longitude and latitude of your GeoJSON
zoom = 17  # Set the zoom level as needed
width, height = 1280, 720  # Set the desired image dimensions
style_id = 'clogxdq2i007r01qr5gmsc907'

# Construct the API request URL
maskurl = f"https://api.mapbox.com/styles/v1/zyjy118//static/geojson({encoded_geojson})/{lon},{lat},{zoom}/{width}x{height}@2x?access_token={access_token}"
url = f"https://api.mapbox.com/styles/v1/mapbox/streets-v11/static/{lon},{lat},{zoom}/{width}x{height}@2x?access_token={access_token}"
# Send the GET request
response1 = requests.get(url)
response2 = requests.get(maskurl)

if response1.status_code == 200:
    # Save the image to a file
    with open('site.png', 'wb') as image_file:
        image_file.write(response1.content)
    print('Image saved successfully!')
else:
    print(f"Failed to retrieve the image: {response1.status_code} - {response1.content}")

if response2.status_code == 200:
    # Save the image to a file
    with open('site_masked.png', 'wb') as image_file:
        image_file.write(response2.content)
    print('Masked Image saved successfully!')
else:
    print(f"Failed to retrieve the masked image: {response2.status_code} - {response2.content}")

simei green
(103.95799378978467, 1.3415044)
[[103.9567511, 1.3409347], [103.9581126, 1.3420741], [103.9581646, 1.3420837], [103.9590268, 1.3408067], [103.9590874, 1.3406294], [103.9576246, 1.3398823], [103.9567511, 1.3409347]]
Image saved successfully!
Failed to retrieve the masked image: 404 - b'{"message":"Not Found"}'


In [5]:
from PIL import Image, ImageEnhance
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [6]:
directory_path = os.getcwd()

mask_image = Image.open(os.path.join(directory_path, 'test_mask_1.jpg'))
original_image = Image.open(os.path.join(directory_path, 'test_gen_1.jpg'))

mask_array = np.array(mask_image)
original_array = np.array(original_image)

# Create mask using blue mask
def create_binary_mask(arr, target_color, threshold=30):
    lower_bound = np.array(target_color) - threshold
    upper_bound = np.array(target_color) + threshold
    mask = (arr[:, :, :3] >= lower_bound) & (arr[:, :, :3] <= upper_bound)
    return np.all(mask, axis=-1)
mask_color = [0, 0, 255]  # bright blue
site_mask = create_binary_mask(mask_array, mask_color)

# Extract site from the original image that corresponds to the blue mask
site_area = original_array.copy()
site_area[~site_mask] = [255, 255, 255]  # making non-masked region white

# Convert site_area back to a PIL image and enhance its contrast
site_area_image = Image.fromarray(site_area)
enhancer = ImageEnhance.Contrast(site_area_image)
enhanced_site = enhancer.enhance(3.0) # amount of contrast
enhanced__site_array = np.array(enhanced_site)

# Combine the enhanced region with the original image
original_contrast_array = original_array.copy()
original_contrast_array[site_mask] = enhanced__site_array[site_mask]
contrast_image = Image.fromarray(original_contrast_array)

# Display the result
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.imshow(original_image)
plt.title('Original Image')
plt.axis('off')
plt.subplot(1, 3, 2)
plt.imshow(mask_image)
plt.title('Masked Image')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(contrast_image)
plt.title('Contrast Image')
plt.axis('off')
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\phuay\\Desktop\\term7_sds\\Evaluate\\test_mask_1.jpg'